In [16]:
import pandas as pd
from getbibleverse import verse_lookup

In [20]:
def store_verses(*passage):

    from string import punctuation
    from heapq import nlargest

    # create empty df to store verses
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    df = pd.DataFrame([])

    # create empty list to store input separately
    books = []
    chapters = []
    verses = []

    # break down input into separate lists
    for entry in passage:
        book, ch_ver = entry.split(" ")
        chapter, verse_range = ch_ver.split(":")

        books.append(book)
        chapters.append(chapter)
        verses.append(verse_range)

    for i in range(len(books)):
        data = pd.json_normalize(verse_lookup(
            books[i], chapters[i], verses[i]))
        verses_df = pd.concat([verses_df, data], ignore_index=True)

    df['book'] = verses_df["bookname"].unique()
    df['chapter'] = verses_df['chapter'].unique()
    df['verses'] = [i for i in verses]
    df['location'] = df["book"] + " " + df["chapter"] + ":" + df["verses"]

    verses_df = verses_df.drop(['chapter', 'verse'], axis=1).groupby(
        'bookname', sort=False).sum().reset_index()

    df['text'] = verses_df['text']

    # NLP


    return 


In [23]:
store_verses("Genesis 1:1-10", "Exodus 3:5-15", "Romans 5:20", "Revelation 4:20-26")
